In [2]:
import pandas as pd

In [3]:
from pyspark.sql import SparkSession
import pyspark
import os
# from dotenv import load_dotenv
# load_dotenv(override=True)
NESSIE_URI = "http://localhost:19120/api/v1"
WAREHOUSE = os.environ.get("WAREHOUSE")  
AWS_ACCESS_KEY = os.environ.get("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.environ.get("AWS_SECRET_KEY")
AWS_S3_ENDPOINT = "http://localhost:9000"
conf = (
    pyspark.SparkConf()
        .setAppName("trade_pipeline")
        .set(
            "spark.jars.packages",
            ",".join([
                "org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1",
                "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0",
                "software.amazon.awssdk:bundle:2.17.178",
                "software.amazon.awssdk:url-connection-client:2.17.178"
            ])
        )
        .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions")
        .set("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
        .set("spark.sql.catalog.nessie.uri", NESSIE_URI)
        .set("spark.sql.catalog.nessie.ref", "main")
        .set("spark.sql.catalog.nessie.authentication.type", "NONE")
        .set("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .set("spark.sql.catalog.nessie.warehouse", WAREHOUSE)
        .set("spark.sql.catalog.nessie.s3.endpoint", AWS_S3_ENDPOINT)
        .set("spark.sql.catalog.nessie.s3.path-style-access", "true")
        .set("spark.sql.catalog.nessie.client.timeout", "30s")
        # S3 Credentials
        .set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY)
        .set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_KEY)
        .set("spark.sql.defaultCatalog", "nessie")
        .set("spark.sql.catalog.demo", "")
)
spark = SparkSession.builder \
        .config(conf=conf) \
        .getOrCreate()

25/09/02 15:04:56 WARN Utils: Your hostname, prahald-ThinkPad-X13-Gen-3 resolves to a loopback address: 127.0.1.1; using 192.168.0.20 instead (on interface wlp0s20f3)
25/09/02 15:04:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/prahald/Documents/Data%20Engineering%20Bootcamp/capstone/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/prahald/.ivy2/cache
The jars for the packages stored in: /home/prahald/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e6a021b4-3917-4662-8e63-56bb6a4d0169;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.3.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12;0.67.0 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in centr

25/09/02 15:04:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/09/02 15:05:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/09/02 15:05:00 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/09/02 15:05:00 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/09/02 15:05:00 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [4]:
spark

In [6]:
# query data from nessie
spark.sql("""select * from nessie.total_export_value_by_port limit 20""").show()

+----+-----+--------+--------------------+----+-------------------+-----------+
|YEAR|MONTH|CTY_CODE|            CTY_NAME|PORT|          PORT_NAME| ALL_VAL_MO|
+----+-----+--------+--------------------+----+-------------------+-----------+
|2015|    3|    null|                ASIA|   -|TOTAL FOR ALL PORTS|40349387606|
|2015|    3|       1|                OPEC|   -|TOTAL FOR ALL PORTS| 6370247872|
|2015|    3|       3|      EUROPEAN UNION|   -|TOTAL FOR ALL PORTS|24132189182|
|2015|    3|      14|PACIFIC RIM COUNT...|   -|TOTAL FOR ALL PORTS|32492696802|
|2015|    3|      17|            CAFTA-DR|   -|TOTAL FOR ALL PORTS| 2491510508|
|2015|    3|      20|       USMCA (NAFTA)|   -|TOTAL FOR ALL PORTS|45395687584|
|2015|    3|      21|TWENTY LATIN AMER...|   -|TOTAL FOR ALL PORTS|32162025588|
|2015|    3|      22|                OECD|   -|TOTAL FOR ALL PORTS|88373181240|
|2015|    3|      23|                NATO|   -|TOTAL FOR ALL PORTS|49463917409|
|2015|    3|      24|               LAFT